# Test

In [9]:
import sys,os
import win32com.client as win32
import time
import datetime
import traceback
import math
import pyspark
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql import HiveContext, SQLContext, SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import ArrayType, FloatType, StringType, StructType, Row, StructField
from pyspark.sql.functions import udf, array, countDistinct

BASE_DIR =r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test'
sys.path.append(BASE_DIR)

from core import configadmin,log_ctrl,color_me

config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
d_set = configadmin.ConfigAdmin(config_path, config_name)
logging = log_ctrl.de8ug_log(logger_name='outlook', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-debug.log'))

# Monitor Alert Check

In [32]:
import sys, os
import datetime, time
import traceback, logging
import pandas as pd
import tkinter
import tkinter.messagebox
import json

BASE_DIR =r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test'
sys.path.append(BASE_DIR)

from pytz import utc
from string import Template
from multiprocessing import Process
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.jobstores.memory import MemoryJobStore
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR, EVENT_JOB_MISSED, EVENT_SCHEDULER_STARTED, EVENT_SCHEDULER_SHUTDOWN
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor
from apscheduler.triggers.cron import CronTrigger

from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core import log_ctrl
from core.outlook import send_email
from core.monitor_check import MonitorCheck

from string import Template
from core.sqlplus import SqlPlus

config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
configs = ConfigAdmin(config_path, config_name)

In [15]:
Job_List = os.path.join(BASE_DIR, 'db', 'Sche', 'MonitorList.xlsx')
df = pd.read_excel(Job_List, 'Sheet1', convert_float=False)
Job_List = df[(df['STATUS']=="A") & (df['SCRIPTS'].isnull() == False) & df.SCRIPTS.apply(lambda x: os.path.exists(os.path.join(BASE_DIR, 'db', 'sql', str(x))) == True)]
Job_List

,NAME,TRIGGER_TYPE,TRIGGER_VALUE,SLEEP_TIME,STATUS,DB,SCRIPTS,SCRIPTS_TYPE,INPUT,CONDITION,NOTIFY,SENTER,TEMPLATE,OUTPUT,PRINCIPAL
5,Nromal Reply Check,CRON,*/10 * 8-21 * * *,28800.0,A,RTDM,Alert_RTDM_0_Normal_Decision.sql,COMPARE,"{""sec"":""900""}",CNT==0,Y,Leo.ZhangZS@homecredit.cn,DecisionLog0Normal.html,USE_CASE,Leo.ZhangZS
6,UseCase Volumn Check,CRON,* 53 */2 * * *,7200.0,A,RTDM,Alert_RTDM_0Data_2h_Decision.sql,COMPARE,"{""sec"":""7200""}",CNT==0,Y,CRMCampaignTeam@homecredit.cn,DecisionLog0Vol.html,RTDM_FLOW_NAME,Leo.ZhangZS


In [24]:
f1 = Job_List[Job_List.index==6]
js = json.loads(f1.to_json(orient='records'))[0]

In [26]:
Rest = MonitorCheck(js).Check()

2020-06-23 17:08:03,338 monitor_check.py [line:150] Main_DWH ERROR Traceback (most recent call last):
  File "C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test\core\monitor_check.py", line 87, in Check
    for i in df.index:
AttributeError: 'NoneType' object has no attribute 'index'



In [40]:
DB = str(js['DB'])
db_user = configs.read_config("SAS_db", f"{DB.lower()}_user")
db_pswd = configs.read_config("SAS_db", f"{DB.lower()}_password")
db_serv = configs.read_config("SAS_db", f"{DB.lower()}_service")

if js['INPUT'] != None:
    INPUT = json.loads(js['INPUT'])
else:
    INPUT = None
with open(os.path.join(BASE_DIR, 'db', 'sql', js['SCRIPTS']), 'r', encoding='utf-8') as f:
    if INPUT != None:
        sql = Template(f.read()).substitute(INPUT)
    else:
        sql = f.read()
    f.close()

In [41]:
print(sql)
res = SqlPlus(db_user, db_pswd, db_serv).fetchall(sql)
res

WITH USECASE AS
 (SELECT DISTINCT LO.RTDM_FLOW_NAME
    FROM APP_RTCDM.CIE_RTDM_DECISION_LOG LO
   WHERE TRUNC(LO.EVENT_DT) > SYSDATE - 3),
CNT AS
 (SELECT LO.RTDM_FLOW_NAME, COUNT(*) AS CNT
    FROM APP_RTCDM.CIE_RTDM_DECISION_LOG LO
   WHERE LO.EVENT_DT + 7200 / (24 * 60 * 60) > SYSDATE
   GROUP BY LO.RTDM_FLOW_NAME)
SELECT A.RTDM_FLOW_NAME, NVL(B.CNT, 0) AS CNT
  FROM USECASE A
  FULL JOIN CNT B
    ON A.RTDM_FLOW_NAME = B.RTDM_FLOW_NAME
 WHERE A.RTDM_FLOW_NAME != 'EPO Service SW_ECAD0KN5NFAUVT4B'
   AND A.RTDM_FLOW_NAME != 'ChatBot WeChat FUP Service_GDEHMLW3QBCGXJRR'


2020-06-23 17:23:26,265 sqlplus.py [line:55] sqlplus INFO 执行成功


{'Status': True,
 'msg':                                                                                        RTDM_FLOW_NAME  \
 ORBP Service_ADCCB3TXSREETZDU                                           ORBP Service_ADCCB3TXSREETZDU   
 AFStuck CAPP FUP Service_HECFZOTAWBH2SPTS                   AFStuck CAPP FUP Service_HECFZOTAWBH2SPTS   
 ChatBot WeChat Service_DCCBJZV3EFE0R024                       ChatBot WeChat Service_DCCBJZV3EFE0R024   
 ContractSigned Service_DGGGJIYSDNAFSOAZ                       ContractSigned Service_DGGGJIYSDNAFSOAZ   
 ExternalTrigger Service_ADHAPYP1XFHTVAPJ                     ExternalTrigger Service_ADHAPYP1XFHTVAPJ   
 OtherOffers FUP Service_HBCBO2DBURFFQRQ0                     OtherOffers FUP Service_HBCBO2DBURFFQRQ0   
 rPOS Client Retention FUP Service_BHAB1OWLJNDZUP11  rPOS Client Retention FUP Service_BHAB1OWLJNDZ...   
 rPOS Client Retention Service_HDAEHWBTKZA0QNBI         rPOS Client Retention Service_HDAEHWBTKZA0QNBI   
 OtherOffers Service_F

In [42]:
import cx_Oracle
db = cx_Oracle.connect(db_user, db_pswd, db_serv)
con = db.cursor()
row = con.execute(sql).fetchall()
cols = [i[0] for i in con.description]
df = pd.DataFrame(data=row, index=[i[0] for i in row], columns=cols)
df

,RTDM_FLOW_NAME,CNT
ORBP Service_ADCCB3TXSREETZDU,ORBP Service_ADCCB3TXSREETZDU,5551
ContractSigned Service_DGGGJIYSDNAFSOAZ,ContractSigned Service_DGGGJIYSDNAFSOAZ,193
ChatBot WeChat Service_DCCBJZV3EFE0R024,ChatBot WeChat Service_DCCBJZV3EFE0R024,135
AFStuck CAPP FUP Service_HECFZOTAWBH2SPTS,AFStuck CAPP FUP Service_HECFZOTAWBH2SPTS,424
ExternalTrigger Service_ADHAPYP1XFHTVAPJ,ExternalTrigger Service_ADHAPYP1XFHTVAPJ,1999
rPOS Client Retention FUP Service_BHAB1OWLJNDZUP11,rPOS Client Retention FUP Service_BHAB1OWLJNDZ...,13304
OtherOffers FUP Service_HBCBO2DBURFFQRQ0,OtherOffers FUP Service_HBCBO2DBURFFQRQ0,638
rPOS Client Retention Service_HDAEHWBTKZA0QNBI,rPOS Client Retention Service_HDAEHWBTKZA0QNBI,51376
OtherOffers Service_FADCW3RNIBBDQ24N,OtherOffers Service_FADCW3RNIBBDQ24N,2071
ChatBot CAPP Service_DCFGUEU5XJCKT2F3,ChatBot CAPP Service_DCFGUEU5XJCKT2F3,248
